# Loop 29 Strategic Analysis

## Current Situation
- Best LB: 70.626088 (exp_024)
- Target: 68.919154
- Gap: 1.707 points (2.47%)
- Our score beats public LB leader (71.191) by 0.567 points

## Key Question: What fundamentally different approaches haven't been tried?

In [1]:
import pandas as pd
import numpy as np
import json
import os

# Load current best submission
df = pd.read_csv('/home/submission/submission.csv')

def parse_s(val):
    if isinstance(val, str) and val.startswith('s'):
        return float(val[1:])
    return float(val)

df['x'] = df['x'].apply(parse_s)
df['y'] = df['y'].apply(parse_s)
df['deg'] = df['deg'].apply(parse_s)
df['n'] = df['id'].apply(lambda x: int(x.split('_')[0]))

print(f"Total rows: {len(df)}")
print(f"N range: {df['n'].min()} to {df['n'].max()}")

Total rows: 20100
N range: 1 to 200


In [2]:
# Calculate score breakdown by N
from shapely.geometry import Polygon
from shapely.affinity import rotate, translate

TREE_TEMPLATE = [
    (0.0, 0.8), (0.125, 0.5), (0.0625, 0.5), (0.2, 0.25), (0.1, 0.25),
    (0.35, 0.0), (0.075, 0.0), (0.075, -0.2), (-0.075, -0.2), (-0.075, 0.0),
    (-0.35, 0.0), (-0.1, 0.25), (-0.2, 0.25), (-0.0625, 0.5), (-0.125, 0.5)
]

def create_tree(x, y, angle):
    tree = Polygon(TREE_TEMPLATE)
    tree = rotate(tree, angle, origin=(0, 0), use_radians=False)
    tree = translate(tree, x, y)
    return tree

def get_n_score(group, n):
    trees = [create_tree(row['x'], row['y'], row['deg']) for _, row in group.iterrows()]
    all_x, all_y = [], []
    for tree in trees:
        minx, miny, maxx, maxy = tree.bounds
        all_x.extend([minx, maxx])
        all_y.extend([miny, maxy])
    side = max(max(all_x) - min(all_x), max(all_y) - min(all_y))
    return (side ** 2) / n, side

scores = {}
side_lengths = {}
for n in range(1, 201):
    group = df[df['n'] == n]
    score, side = get_n_score(group, n)
    scores[n] = score
    side_lengths[n] = side

print(f"Total score: {sum(scores.values()):.6f}")
print(f"Target: 68.919154")
print(f"Gap: {sum(scores.values()) - 68.919154:.6f}")

Total score: 70.624381
Target: 68.919154
Gap: 1.705227


In [3]:
# Analyze score distribution by N ranges
ranges = [(1, 10), (11, 20), (21, 50), (51, 100), (101, 150), (151, 200)]

print("Score breakdown by N range:")
print("="*60)
for start, end in ranges:
    range_score = sum(scores[n] for n in range(start, end+1))
    range_pct = range_score / sum(scores.values()) * 100
    print(f"N={start:3d}-{end:3d}: {range_score:8.4f} ({range_pct:5.2f}%)")

print("="*60)
print(f"Total: {sum(scores.values()):.6f}")

Score breakdown by N range:
N=  1- 10:   4.3291 ( 6.13%)
N= 11- 20:   3.7240 ( 5.27%)
N= 21- 50:  10.9801 (15.55%)
N= 51-100:  17.6119 (24.94%)
N=101-150:  17.1363 (24.26%)
N=151-200:  16.8430 (23.85%)
Total: 70.624381


In [4]:
# Calculate theoretical lower bound (perfect packing efficiency)
# Tree area = 0.2775 (calculated from polygon)
TREE_AREA = 0.2775

print("Theoretical lower bounds (assuming perfect packing):")
print("="*60)
for n in [1, 2, 5, 10, 20, 50, 100, 200]:
    # Perfect packing: N trees fit exactly in a square
    # Area needed = N * TREE_AREA
    # Side = sqrt(N * TREE_AREA)
    # Score contribution = side^2 / N = N * TREE_AREA / N = TREE_AREA
    perfect_side = np.sqrt(n * TREE_AREA)
    perfect_score = (perfect_side ** 2) / n
    actual_score = scores[n]
    efficiency = perfect_score / actual_score * 100
    print(f"N={n:3d}: actual={actual_score:.4f}, perfect={perfect_score:.4f}, efficiency={efficiency:.1f}%")

print("\nNote: Perfect packing is impossible due to tree shape constraints")

Theoretical lower bounds (assuming perfect packing):
N=  1: actual=0.6612, perfect=0.2775, efficiency=42.0%
N=  2: actual=0.4508, perfect=0.2775, efficiency=61.6%
N=  5: actual=0.4168, perfect=0.2775, efficiency=66.6%
N= 10: actual=0.3766, perfect=0.2775, efficiency=73.7%
N= 20: actual=0.3761, perfect=0.2775, efficiency=73.8%
N= 50: actual=0.3608, perfect=0.2775, efficiency=76.9%
N=100: actual=0.3434, perfect=0.2775, efficiency=80.8%
N=200: actual=0.3375, perfect=0.2775, efficiency=82.2%

Note: Perfect packing is impossible due to tree shape constraints


In [5]:
# Identify N values with worst efficiency (most room for improvement)
efficiencies = {}
for n in range(1, 201):
    perfect_side = np.sqrt(n * TREE_AREA)
    perfect_score = (perfect_side ** 2) / n
    actual_score = scores[n]
    efficiencies[n] = perfect_score / actual_score * 100

# Sort by efficiency (lowest = most room for improvement)
sorted_eff = sorted(efficiencies.items(), key=lambda x: x[1])

print("N values with WORST efficiency (most room for improvement):")
print("="*60)
for n, eff in sorted_eff[:20]:
    print(f"N={n:3d}: efficiency={eff:5.1f}%, score={scores[n]:.4f}, side={side_lengths[n]:.4f}")

print("\nN values with BEST efficiency:")
print("="*60)
for n, eff in sorted_eff[-10:]:
    print(f"N={n:3d}: efficiency={eff:5.1f}%, score={scores[n]:.4f}, side={side_lengths[n]:.4f}")

N values with WORST efficiency (most room for improvement):
N=  1: efficiency= 42.0%, score=0.6612, side=0.8132
N=  2: efficiency= 61.6%, score=0.4508, side=0.9495
N=  3: efficiency= 63.8%, score=0.4347, side=1.1420
N=  5: efficiency= 66.6%, score=0.4168, side=1.4437
N=  4: efficiency= 66.6%, score=0.4165, side=1.2908
N=  7: efficiency= 69.4%, score=0.3999, side=1.6731
N=  6: efficiency= 69.4%, score=0.3996, side=1.5484
N=  9: efficiency= 71.6%, score=0.3874, side=1.8673
N=  8: efficiency= 72.0%, score=0.3854, side=1.7559
N= 15: efficiency= 73.6%, score=0.3769, side=2.3779
N= 10: efficiency= 73.7%, score=0.3766, side=1.9407
N= 21: efficiency= 73.7%, score=0.3765, side=2.8117
N= 20: efficiency= 73.8%, score=0.3761, side=2.7425
N= 22: efficiency= 73.9%, score=0.3753, side=2.8733
N= 11: efficiency= 74.0%, score=0.3749, side=2.0308
N= 16: efficiency= 74.2%, score=0.3741, side=2.4466
N= 26: efficiency= 74.2%, score=0.3740, side=3.1183
N= 12: efficiency= 74.5%, score=0.3727, side=2.1149
N= 1

In [6]:
# Calculate how much improvement is needed from each N range to reach target
target = 68.919154
current = sum(scores.values())
gap = current - target

print(f"Current: {current:.6f}")
print(f"Target: {target:.6f}")
print(f"Gap: {gap:.6f}")
print()
print("If we improved each N range by X%, what would be the impact?")
print("="*60)

for start, end in ranges:
    range_score = sum(scores[n] for n in range(start, end+1))
    # How much % improvement needed in this range alone to close the gap?
    pct_needed = (gap / range_score) * 100
    print(f"N={start:3d}-{end:3d}: score={range_score:.4f}, need {pct_needed:.1f}% improvement to close gap")

print()
print("Realistically, we need improvements across ALL N ranges")

Current: 70.624381
Target: 68.919154
Gap: 1.705227

If we improved each N range by X%, what would be the impact?
N=  1- 10: score=4.3291, need 39.4% improvement to close gap
N= 11- 20: score=3.7240, need 45.8% improvement to close gap
N= 21- 50: score=10.9801, need 15.5% improvement to close gap
N= 51-100: score=17.6119, need 9.7% improvement to close gap
N=101-150: score=17.1363, need 10.0% improvement to close gap
N=151-200: score=16.8430, need 10.1% improvement to close gap

Realistically, we need improvements across ALL N ranges


In [7]:
# Check what the jiweiliu kernel claims to achieve
# From the notebook: "It gets ~0.15 improvement in under 2 minutes!"
# Starting from saspav's 71.657 -> 71.46 -> 71.45 -> 71.36

print("jiweiliu kernel claims:")
print("  71.657 -> 71.46 (0.197 improvement)")
print("  71.46 -> 71.45 (0.01 improvement via guided refinement)")
print("  71.45 -> 71.36 (0.09 improvement)")
print()
print(f"Our current: {current:.6f}")
print(f"If we could get similar improvements: {current - 0.3:.6f}")
print(f"Still far from target: {target:.6f}")
print()
print("The jiweiliu approach uses:")
print("  1. Grid-based initial solutions with unit cells")
print("  2. SA optimization with translations")
print("  3. Tree deletion cascade (propagate good configs to smaller N)")
print("  4. Numba acceleration")
print()
print("Key insight: They start from GRID configurations, not random")

jiweiliu kernel claims:
  71.657 -> 71.46 (0.197 improvement)
  71.46 -> 71.45 (0.01 improvement via guided refinement)
  71.45 -> 71.36 (0.09 improvement)

Our current: 70.624381
If we could get similar improvements: 70.324381
Still far from target: 68.919154

The jiweiliu approach uses:
  1. Grid-based initial solutions with unit cells
  2. SA optimization with translations
  3. Tree deletion cascade (propagate good configs to smaller N)
  4. Numba acceleration

Key insight: They start from GRID configurations, not random


In [8]:
# Check what approaches have been tried
experiments_dir = '/home/code/experiments'
experiments = sorted(os.listdir(experiments_dir))

print(f"Total experiments: {len(experiments)}")
print()
print("Approaches tried:")
for exp in experiments:
    print(f"  - {exp}")

Total experiments: 39

Approaches tried:
  - 001_baseline
  - 002_ensemble
  - 003_validated_ensemble
  - 004_bbox3_optimization
  - 005_baseline_validated
  - 005_sa_optimizer
  - 006_zaburo_grid
  - 007_sa_optimization
  - 008_repair_ensemble
  - 009_fractional_translation
  - 010_saspav_best
  - 010_tessellation
  - 011_expanded_ensemble
  - 011_random_restart_sa
  - 012_scanline_packer
  - 013_long_sa
  - 014_basin_hopping
  - 015_constraint_programming
  - 016_rebuild_corners
  - 017_cross_n_extraction
  - 018_egortrushin_tessellation
  - 019_cpp_sa
  - 019_ensemble_n88_improvement
  - 020_asymmetric_solutions
  - 021_snapshot_ensemble
  - 021_tessellation_search
  - 022_best_snapshot_ensemble
  - 022_best_valid_ensemble
  - 022_exhaustive_small_n
  - 022_snapshot_ensemble
  - 023_invalid_snapshot_analysis
  - 024_exhaustive_n1
  - 025_iterative_refinement
  - 026_snapshot_ensemble
  - 026_thorough_cascade
  - 027_iterative_sa_refinement
  - 028_asymmetric_perturbations
  - 029_gr

In [ ]:
# Key insight from research:
# 1. SparroWASM - professional 2D nesting solver (NOT TRIED SYSTEMATICALLY)
# 2. Asymmetric solutions - discussion says they outperform symmetric
# 3. MIP/CP for small N - exact solutions
# 4. High-precision arithmetic (Decimal type)

print("APPROACHES NOT FULLY EXPLORED:")
print("="*60)
print()
print("1. SparroWASM (https://jeroengar.github.io/sparroWASM/)")
print("   - Professional 2D nesting solver")
print("   - sacuscreed kernel shows the workflow")
print("   - Generates fundamentally different solutions")
print("   - NOT systematically tried for N=1-50")
print()
print("2. MIP/CP formulation for small N")
print("   - Exact solver for N=1-10")
print("   - OR-Tools or PuLP")
print("   - NOT tried")
print()
print("3. Different grid configurations")
print("   - jiweiliu uses unit cells with 2 trees")
print("   - Different tessellation patterns")
print("   - NOT fully explored")
print()
print("4. Asymmetric initial configurations")
print("   - Discussion 666880 claims asymmetric beats symmetric")
print("   - Tried but may not have been done correctly")